In [64]:
long_dash = True
fir_like = True

dash_dict = {0 : "Выбрать длинное тире", 1 : "Выбрать короткое тире"}
quotes_dict = {0 : "Выбрать елочки", 1 : "Убрать елочки"}

def reset_dash():
    global long_dash
    long_dash = not long_dash
    butt_dash.config(text = dash_dict[long_dash])
    
    
def reset_quotes():
    global fir_like
    fir_like = not fir_like
    butt_quotes.config(text = quotes_dict[fir_like])
    

In [84]:
import re

def edit_dashes(long_dash, str):
    assert (long_dash in (True, False)), "bool is not bool"
    if long_dash:
        repl = r'— '
    else:
        repl = r'– '
    return re.sub(r'- ', repl, str)

def edit_multispaces(str): #del multiple spaces
    return re.sub(r' {2,}', r' ', str)

def edit_near_quotes(str): # «»&“”-only
    return re.sub(r'([^\.])([,\.])([»”])', r'\1\3\2', str)

def isolate_paragraphs(str): #VK posts only
    return re.sub(r'\n([^–—])', r'\n\n\1', str)

def edit_dialogues(long_dash, str): # -Blahblah -> – Blahblah
    assert (long_dash in (True, False)), "bool is not bool"
    if long_dash:
        repl = r'\n— '
    else:
        repl = r'\n– '
    return re.sub(r'\n-', repl, str)

def edit_quotes(fir_like, str): # «»- those are yolochki, “”- those may work too, "" - this is the browser shit we get rid of
#run this before edit_near_quotes()
    if fir_like:
        left_repl = r'«\1'
        right_repl = r'\1»'
    else:
        left_repl = r'“\1'
        right_repl = r'\1”'
    tmp = re.sub(r'"([\w])', left_repl, str)
    return re.sub(r'([\w\?!,\.])"', right_repl, tmp) 
        
        
def edit_spaces(str): #tricky one
    tmp1 = re.sub(r'([\?!\.])([^\. \n»”])', r'\1 \2', str)
    tmp2 = re.sub(r'([^ ])([«“\(])', r'\1 \2', tmp1)
    tmp3 = re.sub(r'([»”\),])([^ ])', r'\1 \2', tmp2)
    tmp4 = re.sub(r'([«“\(]) ', r'\1', tmp3)
    tmp5 = re.sub(r' ([»”\),\?!])', r'\1', tmp4)
    tmp5 = re.sub(r'([^ \n])([–—])', r'\1 \2', tmp5)
    tmp5 = re.sub(r'([–—])([^ \n])', r'\1 \2', tmp5)
    return re.sub(r' \.([^\.])', r'.\1', tmp5)

In [93]:
def beautify():
    str = field.get("1.0","end-1c")
    str = edit_dashes(long_dash, str)
    str = edit_dialogues(long_dash, str)
    str = edit_multispaces(str)
    str = edit_spaces(str)
    str = isolate_paragraphs(str)
    str = edit_quotes(fir_like, str)
    str = edit_near_quotes(str)
    global result
    result = str
    answer.insert(INSERT, str)
    return str

In [94]:
from tkinter import *
from tkinter import scrolledtext

root = Tk()
root.title("GUI на Python")
root.geometry("1000x250")
 
msg = StringVar()
field = Text(width=100, height = 20)
field.pack()

butt_dash = Button(text = dash_dict[long_dash], command = reset_dash)
butt_dash.pack()

butt_quotes = Button(text = quotes_dict[fir_like], command = reset_quotes)
butt_quotes.pack()
result = ''
editor_butt = Button(text = "Make it better", command = beautify)
editor_butt.pack()

answer = scrolledtext.ScrolledText(width=100, height=20)

answer.pack()
 

root.mainloop()